In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import shutil
import zipfile

# Function to apply augmentations
def augment_image(img, augmentation_type):
    if augmentation_type == 'flip_horizontal':
        return cv2.flip(img, 1)  # Horizontal flip
    elif augmentation_type == 'adjust_contrast':
        return cv2.convertScaleAbs(img, alpha=1.5, beta=0)  # Adjust contrast
    elif augmentation_type == 'rotate':
        rows, cols = img.shape
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), 15, 1)  # Rotate by 15 degrees
        return cv2.warpAffine(img, M, (cols, rows))
    elif augmentation_type == 'zoom':
        zoom_factor = 1.2
        h, w = img.shape[:2]
        new_h, new_w = int(h * zoom_factor), int(w * zoom_factor)
        zoomed_img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        crop_h, crop_w = (new_h - h) // 2, (new_w - w) // 2
        return zoomed_img[crop_h:crop_h + h, crop_w:crop_w + w]
    elif augmentation_type == 'gaussian_noise':
        noise = np.random.normal(0, 25, img.shape).astype(np.uint8)
        return cv2.add(img, noise)
    else:
        return img

# Augmentation types
augmentations = ['flip_horizontal', 'adjust_contrast', 'rotate', 'zoom', 'gaussian_noise']

# Define paths
input_base_path = '/kaggle/input/weakly-labeled-iaaa-dataet/main/abnormal'
output_base_path = '/kaggle/working/abnormal'

# List of MRI types (directories like T1W_SE, T2W_FLAIR, etc.)
mri_types = ['T1W_SE', 'T2W_FLAIR', 'T2W_TSE']

# Loop through each MRI type directory
for mri_type in mri_types:
    mri_path = os.path.join(input_base_path, mri_type)
    
    # Loop through each folder inside the MRI type (each folder contains multiple images)
    for folder_name in os.listdir(mri_path):
        folder_path = os.path.join(mri_path, folder_name)
        
        if os.path.isdir(folder_path):
            # Load all images in the folder
            images = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
            
            for aug_idx, aug_type in enumerate(augmentations):
                # Create a new directory for this augmentation following the input structure
                new_folder_path = os.path.join(output_base_path, mri_type, f"{folder_name}_aug{aug_idx+1}")
                os.makedirs(new_folder_path, exist_ok=True)
                
                # Apply augmentation to each image
                for img_name in tqdm(images, desc=f"Processing {folder_name} with {aug_type}"):
                    img_path = os.path.join(folder_path, img_name)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    
                    # Apply the augmentation
                    aug_img = augment_image(img, aug_type)
                    
                    # Save the augmented image to the new directory
                    new_img_path = os.path.join(new_folder_path, img_name)
                    cv2.imwrite(new_img_path, aug_img)

# Create a zip file of the augmented images
zip_file_path = '/kaggle/working/augmented_images.zip'
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, dirs, files in os.walk(output_base_path):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_base_path))

# Remove the output base path to clean up
shutil.rmtree(output_base_path)

Processing 1.3.46.670589.11.10042.5.0.4724.2024031721105914163 with flip_horizontal: 100%|██████████| 16/16 [00:00<00:00, 120.25it/s]
Processing 1.3.46.670589.11.10042.5.0.4724.2024031721105914163 with adjust_contrast: 100%|██████████| 16/16 [00:00<00:00, 579.20it/s]
Processing 1.3.46.670589.11.10042.5.0.4724.2024031721105914163 with rotate: 100%|██████████| 16/16 [00:00<00:00, 409.84it/s]
Processing 1.3.46.670589.11.10042.5.0.4724.2024031721105914163 with zoom: 100%|██████████| 16/16 [00:00<00:00, 337.91it/s]
Processing 1.3.46.670589.11.10042.5.0.4724.2024031721105914163 with gaussian_noise: 100%|██████████| 16/16 [00:00<00:00, 204.32it/s]
Processing 1.3.46.670589.11.10042.5.0.6596.2024022209121454072 with flip_horizontal: 100%|██████████| 16/16 [00:00<00:00, 137.32it/s]
Processing 1.3.46.670589.11.10042.5.0.6596.2024022209121454072 with adjust_contrast: 100%|██████████| 16/16 [00:00<00:00, 561.61it/s]
Processing 1.3.46.670589.11.10042.5.0.6596.2024022209121454072 with rotate: 100%|██